In [ ]:
import gzip
import os
import pandas as pd
from io import StringIO
import numpy as np
import datetime
import dateutil.parser as parser

def read_stock_data(data_home, data_type, venue, year, month, day, ticker, is_filter):
    '''
    从sever中读取一只股票一天的数据
    data_home:数据所在folder
    data_type:类型
    venue:交易所
    is_filter:是否进行filter操作
    '''

    path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
        month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue)  # 读数据的路径

    if (os.path.exists(path)):
        data0 = open(path, 'r')
        data1 = pd.read_csv(StringIO(data0.read()))
        data = data1.loc[:, ['time', 'volume', 'current','a1_v','a1_p','b1_v','b1_p',]]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        data['spread'] = (data['a1_p'] - data['b1_p'])/(data['a1_p'] + data['b1_p'])
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]         # 删除大于99.9%分位数的数据
            return data
    else:
        path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
            month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue) + '.gz'
        with gzip.open(path, 'rb') as gf:
            data1 = pd.read_csv(gf)
        data = data1.loc[:, ['time', 'volume','current', 'a1_v','b1_v']]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]  # 删除大于99.9%分位数的数据
            return data

data = pd.read_csv('/Users/lihaohan/Desktop/000725-2.csv')

df = pd.DataFrame(data)
def convert_time(time_str):
    # 解析时间字符串，获取年、月、日等信息
    time_str = str(time_str)
    year = int(time_str[0:4])
    month = int(time_str[4:6])
    day = int(time_str[6:8])
    hour = int(time_str[8:10])
    minute = int(time_str[10:12])
    second = int(time_str[12:14])

   # 创建datetime对象并返回
    return datetime.datetime(year, month, day, hour, minute, second)
df['time'] = df['time'].apply(lambda x: convert_time(x))
df['date'] = df['time'].dt.date
filtered_data = data[data['time'].dt.time < datetime.time(14, 57)]

filtered_data.sort_values(['time','date', 'volume'], inplace=True)

filtered_data['quantile_999'] = filtered_data['volume'].transform(lambda x: x.rolling(window=24070).max().shift(1)*0.999)

filtered_data['winsorized_volume'] = np.where(filtered_data['volume'] > filtered_data['quantile_999'], filtered_data['quantile_999'], filtered_data['volume'])

print(filtered_data)
filtered_data.to_csv('/Users/lihaohan/Desktop/000725.csv', index=False)

In [ ]:
def trans_date(date):  # transform date to year-month-day
    dates = []
    for i in range(len(date)):
        year = str(date[i])[0:4]
        month = str(date[i])[4:6]
        day = str(date[i])[6:8]
        date_std = datetime.date(int(year), int(month), int(day)).isoformat()
        dates.append(date_std)
    return dates


def trans_time(time):  # transform time to hour:minute:second
    times = []
    for i in range(len(time)):
        hour = str(time[i])[8:10]
        minute = str(time[i])[10:12]
        second = str(time[i])[12:14]
        time_std = datetime.time(int(hour), int(minute), int(second)).isoformat()
        times.append(time_std)
    return times

def divide_bin(time, binnumber):  # 计算每条交易所属的bin number
    '''
    time:columns of time
    '''
    n = 237 / (binnumber - 1) * 60
    bin_nums = []
    for i in range(len(time)):
        if datetime.datetime.strptime(time[i],"%H:%M:%S") < datetime.datetime.strptime("09:30:00","%H:%M:%S"):
            bin_num = 0  # 交易发生在9：30之前，bin number为0
        elif datetime.datetime.strptime(time[i],"%H:%M:%S")>datetime.datetime.strptime("15:00:00","%H:%M:%S"):
            bin_num = binnumber+1
        else:
            starttime = parser.parse(datetime.time(9, 30, 0).isoformat())  # 开始时间设为9：30
            endtime = parser.parse(time[i])  # 结束时间是该条数据的交易时间
            s = (endtime - starttime).seconds  # 从开盘到现在的秒数
            if s > -1 and s < 7201:  # 交易发生在9：30-11：30之前
                bin_num = int((s - 0.5) // n) + 1  # 9：30之后的bin number从1开始
            elif s > 12599 and s < 19801:  # 13:00-15:00
                bin_num = int((s - 0.5 - 5400) // n)+1  # 去掉中间的90分钟
            else:
                bin_num = binnumber
        bin_nums.append(bin_num)
    return bin_nums

####### volatility、quote imbalance、spread 
def cal_bin_volume(subdf, binnumber):
    '''
    subdf: data to be processed, DataFrame
    return: DataFrame including one stock, ranked by bin number
    '''

    subdf = subdf[~subdf['bin_num'].isin([binnumber + 1])]  # Exclude rows with bin number binnumber+1
    subdf = subdf[~subdf['bin_num'].isin([binnumber])]  # Exclude rows with bin number binnumber

    daily_volume = subdf['volume'].groupby(subdf['date']).sum().reset_index()  # Calculate total volume for each day
    bin_volume = subdf['volume'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()  # Calculate volume for each bin
    
    # 计算标准差
    
    volatility = subdf['volume'].groupby([subdf['date'], subdf['bin_num']]).std().reset_index().rename(columns={'volume': 'volatility'})


    # 一档买卖价的数量
    bin_ask = subdf['a1_v'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()
    bin_bid = subdf['b1_v'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()
    
    # 定义计算几何平均的函数 （有0和-1）

    def geometric_mean(data):
        # 对数据加1
        data_add1 = data + 1

        # 将0值替换为非零小值
        data_add1[data_add1 == 0] = 1e-9

        # 计算几何平均
        geometric_mean = np.exp(np.mean(np.log(data_add1)))

        # 减去1得到最终结果
        result = geometric_mean - 1

        return result



    # 按日期分组，并计算每个组的几何平均
    spread = subdf['spread'].groupby([subdf['date'], subdf['bin_num']]).apply(geometric_mean).reset_index()
    df = pd.merge(daily_volume, bin_volume, how='outer', on='date')  # Merge daily_volume and bin_volume
    subdf1 = pd.merge(df, spread,how='outer',on=['date','bin_num'])
    
    # imbalance
    subdf1['quote_imbalance'] = (bin_bid['b1_v'] - bin_ask['a1_v']) / (bin_bid['b1_v'] + bin_ask['a1_v'])
    return subdf1

In [27]:
import gzip
import os
import pandas as pd
from io import StringIO
import numpy as np
import datetime
import dateutil.parser as parser
import pandas_market_calendars as mcal
data_home = '/volume1/sinoalgo/data/sinoalgo/JQMarketData'
data_type = 'STOCK'
venue= 'XSHE'
ticker = '000725'
start_date = "2020-09-01"
end_date = "2021-06-30"
bin_num = 25

def read_stock_data(data_home, data_type, venue, year, month, day, ticker, is_filter):
    '''
    从sever中读取一只股票一天的数据
    data_home:数据所在folder
    data_type:类型
    venue:交易所
    is_filter:是否进行filter操作
    '''

    path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
        month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue)  # 读数据的路径

    if (os.path.exists(path)):
        data0 = open(path, 'r')
        data1 = pd.read_csv(StringIO(data0.read()))
        data = data1.loc[:, ['time', 'volume', 'current','a1_v','a1_p','b1_v','b1_p',]]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        data['spread'] = (data['a1_p'] - data['b1_p'])/(data['a1_p'] + data['b1_p'])
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]         # 删除大于99.9%分位数的数据
            return data
    else:
        path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
            month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue) + '.gz'
        with gzip.open(path, 'rb') as gf:
            data1 = pd.read_csv(gf)
        data = data1.loc[:, ['time', 'volume','current', 'a1_v','b1_v']]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]  # 删除大于99.9%分位数的数据
            return data

# 将日期格式化为年份、月份和日份的形式
start_year, start_month, start_day = start_date.split('-')
end_year, end_month, end_day = end_date.split('-')

# 创建一个空的DataFrame来存储所有数据
all_data = pd.DataFrame()

# 获取交易日历
cal = mcal.get_calendar('SSE')
schedule = cal.schedule(start_date=start_date, end_date=end_date)

# 遍历交易日历并导出每个交易日的数据
for d in schedule.index:
    date_str = d.strftime('%Y-%m-%d')
    date = d.to_pydatetime()
    year,month,day = date_str.split('-')
    
# 判断日期是否是周末或节假日
    if cal.valid_days(date, end_date).empty:
        continue
        
    # 读取指定日期的数据
    data = read_stock_data(data_home, data_type, venue, year, month, day, ticker, 0)
    
    # 将数据追加到总数据中
    all_data = all_data.append(data)
            
df = all_data

def convert_time(time_str):
    # 解析时间字符串，获取年、月、日等信息
    time_str = str(time_str)
    year = int(time_str[0:4])
    month = int(time_str[4:6])
    day = int(time_str[6:8])
    hour = int(time_str[8:10])
    minute = int(time_str[10:12])
    second = int(time_str[12:14])

   # 创建datetime对象并返回
    return datetime.datetime(year, month, day, hour, minute, second)
df['time'] = df['time'].apply(lambda x: convert_time(x))
df['date'] = df['time'].dt.date
filtered_data = df[df['time'].dt.time < datetime.time(14, 57)]

filtered_data.sort_values(['time','date', 'volume'], inplace=True)

filtered_data['quantile_999'] = filtered_data['volume'].transform(lambda x: x.rolling(window=24070).max().shift(1)*0.999)

filtered_data['winsorized_volume'] = np.where(filtered_data['volume'] > filtered_data['quantile_999'], filtered_data['quantile_999'], filtered_data['volume'])

df = filtered_data
df['time'] = df['time'].apply(lambda x: x.strftime('%Y%m%d%H%M%S'))

def trans_date(date):  # transform date to year-month-day
    dates = []
    for i in range(len(date)):
        year = str(date[i])[0:4]
        month = str(date[i])[4:6]
        day = str(date[i])[6:8]
        date_std = datetime.date(int(year), int(month), int(day)).isoformat()
        dates.append(date_std)
    return dates


def trans_time(time):  # transform time to hour:minute:second
    times = []
    for i in range(len(time)):
        hour = str(time[i])[8:10]
        minute = str(time[i])[10:12]
        second = str(time[i])[12:14]
        time_std = datetime.time(int(hour), int(minute), int(second)).isoformat()
        times.append(time_std)
    return times

def divide_bin(time, binnumber):  # 计算每条交易所属的bin number
    '''
    time:columns of time
    '''
    n = 237 / (binnumber - 1) * 60
    bin_nums = []
    for i in range(len(time)):
        if datetime.datetime.strptime(str(time[i]),"%H:%M:%S") < datetime.datetime.strptime("09:30:00","%H:%M:%S"):
            bin_num = 0  # 交易发生在9：30之前，bin number为0
        elif datetime.datetime.strptime(str(time[i]),"%H:%M:%S")>datetime.datetime.strptime("15:00:00","%H:%M:%S"):

            bin_num = binnumber+1
        else:
            starttime = parser.parse(datetime.time(9, 30, 0).isoformat())  # 开始时间设为9：30
            endtime = parser.parse(time[i])  # 结束时间是该条数据的交易时间
            s = (endtime - starttime).seconds  # 从开盘到现在的秒数
            if s > -1 and s < 7201:  # 交易发生在9：30-11：30之前
                bin_num = int((s - 0.5) // n) + 1  # 9：30之后的bin number从1开始
            elif s > 12599 and s < 19801:  # 13:00-15:00
                bin_num = int((s - 0.5 - 5400) // n)+1  # 去掉中间的90分钟
            else:
                bin_num = binnumber
        bin_nums.append(bin_num)
    return bin_nums

####### volatility、quote imbalance、spread 
def cal_bin_volume(subdf, binnumber):
    '''
    subdf: data to be processed, DataFrame
    return: DataFrame including one stock, ranked by bin number
    '''

    subdf = subdf[~subdf['bin_num'].isin([binnumber + 1])]  # Exclude rows with bin number binnumber+1
    subdf = subdf[~subdf['bin_num'].isin([binnumber])]  # Exclude rows with bin number binnumber

    daily_volume = subdf['winsorized_volume'].groupby(subdf['date']).sum().reset_index()  # Calculate total volume for each day
    bin_volume = subdf['winsorized_volume'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()  # Calculate volume for each bin
    
    # 计算标准差
    
    volatility = subdf['winsorized_volume'].groupby([subdf['date'], subdf['bin_num']]).std().reset_index().rename(columns={'volume': 'volatility'})


    # 一档买卖价的数量
    bin_ask = subdf['a1_v'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()
    bin_bid = subdf['b1_v'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()

    # 定义计算几何平均的函数 （有0和-1）

    def geometric_mean(data):
        # 对数据加1
        data_add1 = data + 1

        # 将0值替换为非零小值
        data_add1[data_add1 == 0] = 1e-9

        # 计算几何平均
        geometric_mean = np.exp(np.mean(np.log(data_add1)))

        # 减去1得到最终结果
        result = geometric_mean - 1

        return result
    # 按日期分组，并计算每个组的几何平均
    spread = vol_df['spread'].groupby([vol_df['time'], vol_df['bin_num']]).apply(geometric_mean).reset_index()
    df = pd.merge(daily_volume, bin_volume, how='outer', on='date')  # Merge daily_volume and bin_volume
    subdf1 = pd.merge(df, spread,how='outer',on=['date','bin_num'])
    
    # imbalance
    subdf1['quote_imbalance'] = (bin_bid['b1_v'] - bin_ask['a1_v']) / (bin_bid['b1_v'] + bin_ask['a1_v'])
    return subdf1
    
df = pd.DataFrame(df)
df = df.reset_index(drop=True)
transdate = trans_date(df['time'])
transtime = trans_time(df['time'])
df.loc[:, 'date'] = transdate  # replace the original data by transformed data
df.loc[:, 'time'] = transtime
bin_nums = divide_bin(time=df['date'], binnumber=bin_num)
df.loc[:, 'bin_num'] = bin_nums
vol_df = cal_bin_volume(subdf=df, binnumber=bin_num)
vol_df = vol_df.rename(columns={'volume_x': 'daily_volume'})
vol_df = vol_df.rename(columns={'volume_y': 'bin_volume'})
vol_df['bin_volume'] = vol_df['bin_volume'].fillna(1)  # 空值用1填充
vol_df['daily_volume'] = vol_df['daily_volume'].fillna(method = 'bfill')  # 空值用向上填充


    
filename_basic = 'LHH' + str(ticker) + '_' + str(venue) + '_' + str(bin_num) +'_'+'daily.csv'
vol_df.to_csv(filename_basic, index=False)

/tmp/ipykernel_1151102/1271722643.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1151102/1271722643.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1151102/1271722643.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1151102/1271722643.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1151102/1271722643.py:84: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

TypeError: 'NoneType' object is not subscriptable

In [17]:
    df = df.reset_index(drop=True)
    transdate = trans_date(df['time'])
    transtime = trans_time(df['time'])
    df.loc[:, 'date'] = transdate  # replace the original data by transformed data
    df.loc[:, 'timet'] = transtime
    bin_nums = divide_bin(time=df['timet'], binnumber=bin_num)
    df.loc[:, 'bin_num'] = bin_nums
    vol_df = cal_bin_volume(subdf=df, binnumber=bin_num)
    vol_df = vol_df.rename(columns={'volume_x': 'daily_volume'})
    vol_df = vol_df.rename(columns={'volume_y': 'bin_volume'})
    vol_df['bin_volume'] = vol_df['bin_volume'].fillna(1)  # 空值用1填充
    vol_df['daily_volume'] = vol_df['daily_volume'].fillna(method = 'bfill')  # 空值用向上填充
    return vol_df

ValueError: invalid literal for int() with base 10: '9-'

In [2]:
import gzip
import os
import pandas as pd
from io import StringIO
import numpy as np
import datetime
from datetime import datetime
import dateutil.parser as parser
import pandas_market_calendars as mcal
data_home = '/volume1/sinoalgo/data/sinoalgo/JQMarketData'
data_type = 'STOCK'
venue= 'XSHE'
ticker = '000725'
start_date = "2020-09-01"
end_date = "2021-06-30"
bin_num = 25

def read_stock_data(data_home, data_type, venue, year, month, day, ticker, is_filter):
    '''
    从sever中读取一只股票一天的数据
    data_home:数据所在folder
    data_type:类型
    venue:交易所
    is_filter:是否进行filter操作
    '''

    path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
        month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue)  # 读数据的路径

    if (os.path.exists(path)):
        data0 = open(path, 'r')
        data1 = pd.read_csv(StringIO(data0.read()))
        data = data1.loc[:, ['time', 'volume', 'current','a1_v','a1_p','b1_v','b1_p',]]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        data['spread'] = (data['a1_p'] - data['b1_p'])/(data['a1_p'] + data['b1_p'])
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]         # 删除大于99.9%分位数的数据
            return data
    else:
        path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
            month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue) + '.gz'
        with gzip.open(path, 'rb') as gf:
            data1 = pd.read_csv(gf)
        data = data1.loc[:, ['time', 'volume','current', 'a1_v','b1_v']]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]  # 删除大于99.9%分位数的数据
            return data

# 将日期格式化为年份、月份和日份的形式
start_year, start_month, start_day = start_date.split('-')
end_year, end_month, end_day = end_date.split('-')

# 创建一个空的DataFrame来存储所有数据
all_data = pd.DataFrame()

# 获取交易日历
cal = mcal.get_calendar('SSE')
schedule = cal.schedule(start_date=start_date, end_date=end_date)

# 遍历交易日历并导出每个交易日的数据
for d in schedule.index:
    date_str = d.strftime('%Y-%m-%d')
    date = d.to_pydatetime()
    year,month,day = date_str.split('-')
    
# 判断日期是否是周末或节假日
    if cal.valid_days(date, end_date).empty:
        continue
        
    # 读取指定日期的数据
    data = read_stock_data(data_home, data_type, venue, year, month, day, ticker, 0)
    
    # 将数据追加到总数据中
    all_data = all_data.append(data)
            
df = all_data.drop(date,axis=1)
df['time'] = datetime.strptime(df['time'], "%Y-%m-%d %H:%M:%S")
df['time'] = df['time'].strftime("%Y%m%d%H%M%S")



def convert_time(time_str):
    # 解析时间字符串，获取年、月、日等信息
    time_str = str(time_str)
    year = int(time_str[0:4])
    month = int(time_str[4:6])
    day = int(time_str[6:8])
    hour = int(time_str[8:10])
    minute = int(time_str[10:12])
    second = int(time_str[12:14])

   # 创建datetime对象并返回
    return datetime.datetime(year, month, day, hour, minute, second)
df['time'] = df['time'].apply(lambda x: convert_time(x))
df['date'] = df['time'].dt.date
filtered_data = df[df['time'].dt.time < datetime.time(14, 57)]

filtered_data.sort_values(['time','date', 'volume'], inplace=True)

filtered_data['quantile_999'] = filtered_data['volume'].transform(lambda x: x.rolling(window=24070).max().shift(1)*0.999)

filtered_data['winsorized_volume'] = np.where(filtered_data['volume'] > filtered_data['quantile_999'], filtered_data['quantile_999'], filtered_data['volume'])

df = filtered_data


def trans_date(date):  # transform date to year-month-day
    dates = []
    for i in range(len(date)):
        year = str(date[i])[0:4]
        month = str(date[i])[4:6]
        day = str(date[i])[6:8]
        date_std = datetime.date(int(year), int(month), int(day)).isoformat()
        dates.append(date_std)
    return dates


def trans_time(time):  # transform time to hour:minute:second
    times = []
    for i in range(len(time)):
        hour = str(time[i])[8:10]
        minute = str(time[i])[10:12]
        second = str(time[i])[12:14]
        time_std = datetime.time(int(hour), int(minute), int(second)).isoformat()
        times.append(time_std)
    return times

def divide_bin(time, binnumber):  # 计算每条交易所属的bin number
    '''
    time:columns of time
    '''
    n = 237 / (binnumber - 1) * 60
    bin_nums = []
    for i in range(len(time)):
        if datetime.datetime.strptime(str(time[i]),"%H:%M:%S") < datetime.datetime.strptime("09:30:00","%H:%M:%S"):
            bin_num = 0  # 交易发生在9：30之前，bin number为0
        elif datetime.datetime.strptime(str(time[i]),"%H:%M:%S")>datetime.datetime.strptime("15:00:00","%H:%M:%S"):

            bin_num = binnumber+1
        else:
            starttime = parser.parse(datetime.time(9, 30, 0).isoformat())  # 开始时间设为9：30
            endtime = parser.parse(time[i])  # 结束时间是该条数据的交易时间
            s = (endtime - starttime).seconds  # 从开盘到现在的秒数
            if s > -1 and s < 7201:  # 交易发生在9：30-11：30之前
                bin_num = int((s - 0.5) // n) + 1  # 9：30之后的bin number从1开始
            elif s > 12599 and s < 19801:  # 13:00-15:00
                bin_num = int((s - 0.5 - 5400) // n)+1  # 去掉中间的90分钟
            else:
                bin_num = binnumber
        bin_nums.append(bin_num)
    return bin_nums


####### volatility、quote imbalance、spread 
def cal_bin_volume(subdf, binnumber):
    '''
    subdf: data to be processed, DataFrame
    return: DataFrame including one stock, ranked by bin number
    '''

    subdf = subdf[~subdf['bin_num'].isin([binnumber + 1])]  # Exclude rows with bin number binnumber+1
    subdf = subdf[~subdf['bin_num'].isin([binnumber])]  # Exclude rows with bin number binnumber

    daily_volume = subdf['winsorized_volume'].groupby(subdf['date']).sum().reset_index()  # Calculate total volume for each day
    bin_volume = subdf['winsorized_volume'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()  # Calculate volume for each bin
    
    # 计算标准差
    
    volatility = subdf['winsorized_volume'].groupby([subdf['date'], subdf['bin_num']]).std().reset_index().rename(columns={'volume': 'volatility'})


    # 一档买卖价的数量
    bin_ask = subdf['a1_v'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()
    bin_bid = subdf['b1_v'].groupby([subdf['date'], subdf['bin_num']]).sum().reset_index()
    
    # 定义计算几何平均的函数 （有0和-1）

    def geometric_mean(data):
        # 对数据加1
        data_add1 = data + 1

        # 将0值替换为非零小值
        data_add1[data_add1 == 0] = 1e-9

        # 计算几何平均
        geometric_mean = np.exp(np.mean(np.log(data_add1)))

        # 减去1得到最终结果
        result = geometric_mean - 1

        return result


df = pd.DataFrame(df)
df = df.reset_index(drop=True)
transdate = trans_date(df['time'])
transtime = trans_time(df['time'])
df.loc[:, 'date'] = transdate  # replace the original data by transformed data
df.loc[:, 'time'] = transtime
bin_nums = divide_bin(time=df['time'], binnumber=bin_num)
df.loc[:, 'bin_num'] = bin_nums
vol_df = cal_bin_volume(subdf=df, binnumber=bin_num)
vol_df = vol_df.rename(columns={'volume_x': 'daily_volume'})
vol_df = vol_df.rename(columns={'volume_y': 'bin_volume'})
vol_df['bin_volume'] = vol_df['bin_volume'].fillna(1)  # 空值用1填充
vol_df['daily_volume'] = vol_df['daily_volume'].fillna(method = 'bfill')  # 空值用向上填充



# 按日期分组，并计算每个组的几何平均
spread = subdf['spread'].groupby([subdf['date'], subdf['bin_num']]).apply(geometric_mean).reset_index()
df = pd.merge(daily_volume, bin_volume, how='outer', on='date')  # Merge daily_volume and bin_volume
subdf1 = pd.merge(df, spread,how='outer',on=['date','bin_num'])
    
# imbalance
subdf1['quote_imbalance'] = (bin_bid['b1_v'] - bin_ask['a1_v']) / (bin_bid['b1_v'] + bin_ask['a1_v'])
return subdf1

filename_basic = 'LHH' + str(ticker) + '_' + str(venue) + '_' + str(bin_num) +'_'+'daily.csv'
df.to_csv(filename_basic, index=False)

/tmp/ipykernel_1232910/1167796702.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1232910/1167796702.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1232910/1167796702.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1232910/1167796702.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(data)
/tmp/ipykernel_1232910/1167796702.py:85: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat

KeyError: '[datetime.datetime(2021, 6, 30, 0, 0)] not found in axis'